In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
brand=pd.read_csv("train_brand.csv")
category=pd.read_csv("train_category.csv")
company=pd.read_csv("train_company.csv")

In [3]:
consumer=pd.read_csv("consumer_total.csv")
train_History=pd.read_csv("trainHistory.csv")

In [4]:
offers=pd.read_csv("offers.csv")

In [5]:
offers.head()

,offer,category,quantity,company,offervalue,brand
0,1190530,9115,1,108500080,5.00,93904
1,1194044,9909,1,107127979,1.00,6732
2,1197502,3203,1,106414464,0.75,13474
3,1198271,5558,1,107120272,1.50,5072
4,1198272,5558,1,107120272,1.50,5072


In [6]:
train_History[train_History.id==1000714152]

,id,chain,offer,market,repeattrips,repeater,offerdate
81550,1000714152,46,1199258,15,1,t,2013-03-15


In [7]:
train_offer=train_History.merge(offers,on='offer')
train_offer.head()

,id,chain,offer,market,repeattrips,repeater,offerdate,category,quantity,company,offervalue,brand
0,86246,205,1208251,34,5,t,2013-04-24,2202,1,104460040,2.0,3718
1,15753725,17,1208251,4,0,f,2013-04-24,2202,1,104460040,2.0,3718
2,16535563,4,1208251,1,7,t,2013-04-27,2202,1,104460040,2.0,3718
3,18259179,3,1208251,2,0,f,2013-04-24,2202,1,104460040,2.0,3718
4,21024070,15,1208251,9,1,t,2013-04-23,2202,1,104460040,2.0,3718


In [8]:
trainconsumer=consumer.merge(train_offer,on="id")
trainconsumer.head()

,id,total_amount,total_quantity,total_trips,diffcompany,diffcategory,diffbrand,chain,offer,market,repeattrips,repeater,offerdate,category,quantity,company,offervalue,brand
0,100051423,2771.429977,807,85,190,192,228,4,1197502,1,0,f,2013-03-26,3203,1,106414464,0.75,13474
1,100084808,2790.199984,907,74,221,213,249,20,1197502,7,0,f,2013-03-26,3203,1,106414464,0.75,13474
2,100172712,7690.499942,3349,222,320,317,381,95,1200581,39,0,f,2013-03-30,1726,1,104460040,1.25,7668
3,100200838,3880.729944,1001,61,192,170,222,17,1208251,4,0,f,2013-04-24,2202,1,104460040,2.00,3718
4,1002116097,4158.509984,1612,61,206,228,266,46,1197502,15,1,t,2013-03-30,3203,1,106414464,0.75,13474


In [9]:
def diff_days(s1,s2):
    date_format = "%Y/%m/%d"
    a = datetime.datetime.strptime(s1, date_format)
    b = datetime.datetime.strptime(s2, date_format)
    delta = b - a
    return delta.days

In [10]:
def diff_offdays(s1,s2):
    a = datetime.datetime.strptime(s1, "%Y/%m/%d")
    b = datetime.datetime.strptime(s2, "%Y-%m-%d")
    delta = b - a
    return delta.days

In [11]:
consumer_brand=pd.DataFrame()
consumer_brand['id']=brand.id.unique()
consumer_category=pd.DataFrame()
consumer_category['id']=category.id.unique()
consumer_company=pd.DataFrame()
consumer_company['id']=company.id.unique()


In [12]:
#bybrand
def extrafeature(x,table):
    print('start-',x)
    if x=='brand':
        varx=brand
    elif x =='category':
        varx=category
    else:
        varx=company
    unique_id=varx.id.unique()
    for i in range(len(unique_id)):
        selectdata=varx[varx['id']== unique_id[i]].reset_index()
        offerday=trainconsumer.loc[trainconsumer.id ==selectdata.id[0],'offerdate'].values[0] 
        table.loc[i,'id']=unique_id[i]
        table.loc[i,x]=selectdata.loc[0,x]
        times=selectdata.shape[0]
        table.loc[i,'purchasetimes_'+x]=times
        table.loc[i,'total_amount_'+x]=selectdata['amount'].sum()
        table.loc[i,'total_quantit_'+x]=selectdata['quantity'].sum()
        table.loc[i,'purchaseperiod_'+x]=0
        table.loc[i,x+'period']=0
        table.loc[i,'before_30day_time_'+x]=0
        table.loc[i,'before_30day_quan_'+x]=0
        table.loc[i,'before_30day_amount_'+x]=0
        table.loc[i,'before_60day_time_'+x]=0
        table.loc[i,'before_60day_quan_'+x]=0
        table.loc[i,'before_60day_amount_'+x]=0
        table.loc[i,'before_90day_time_'+x]=0
        table.loc[i,'before_90day_quan_'+x]=0
        table.loc[i,'before_90day_amount_'+x]=0
        table.loc[i,'before_180day_time_'+x]=0
        table.loc[i,'before_180day_quan_'+x]=0
        table.loc[i,'before_180day_amount_'+x]=0
     
        
        if times >1:
            sumday=0
            period=0
            for k in range(1,times):
                offergap=diff_offdays(selectdata.loc[k,'date'],offerday)
                timegap=diff_days(selectdata.loc[k-1,'date'],selectdata.loc[k,'date'])
                sumday=sumday+int(timegap)
                period=period+(int(timegap)/(selectdata.loc[k-1,'quantity']))
                if  0 < offergap < 30:
                    table.loc[i,'before_30day_time_'+x]+= 1.0
                    table.loc[i,'before_30day_quan_'+x]+= float(selectdata.loc[k,'quantity'])
                    table.loc[i,'before_30day_amount_'+x]+= float(selectdata.loc[k,'amount'])
                        
                if 0 < offergap < 60:
                    table.loc[i,'before_60day_time_'+x]+= 1.0
                    table.loc[i,'before_60day_quan_'+x]+= float(selectdata.loc[k,'quantity'])
                    table.loc[i,'before_60day_amount_'+x]+= float(selectdata.loc[k,'amount'])
                        
                if  0 < offergap < 90:
                    table.loc[i,'before_90day_time_'+x]+= 1.0
                    table.loc[i,'before_90day_quan_'+x]+= float(selectdata.loc[k,'quantity'])
                    table.loc[i,'before_90day_amount_'+x]+= float(selectdata.loc[k,'amount'])
                        
                if 0 <  offergap < 180:
                    table.loc[i,'before_180day_time_'+x]+= 1.0
                    table.loc[i,'before_180day_quan_'+x]+= float(selectdata.loc[k,'quantity'])
                    table.loc[i,'before_180day_amount_'+x]+= float(selectdata.loc[k,'amount'])
                           
            if type(sumday)==int and sumday>0:    
                table.loc[i,'purchaseperiod_'+x]=(sumday/(times-1))
            if type(period)==int and period>0:
                table.loc[i,x+'period']=(period/(times-1))
        if i%10000==0:
            print(x+": "+ str(i)+"/"+str(len(unique_id)))
            
  

        

In [13]:
extrafeature('brand',consumer_brand)
extrafeature('company',consumer_company)
extrafeature('category',consumer_category)


start- brand
brand: 0/63151


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in long_scalars


brand: 10000/63151
brand: 20000/63151
brand: 30000/63151
brand: 40000/63151
brand: 50000/63151
brand: 60000/63151
start- company
company: 0/86741
company: 10000/86741
company: 20000/86741
company: 30000/86741
company: 40000/86741
company: 50000/86741
company: 60000/86741
company: 70000/86741
company: 80000/86741
start- category
category: 0/87541
category: 10000/87541
category: 20000/87541
category: 30000/87541
category: 40000/87541
category: 50000/87541
category: 60000/87541
category: 70000/87541
category: 80000/87541


In [14]:
consumer_brand.to_csv("consumer_brand.csv")
consumer_company.to_csv("consumer_company.csv")
consumer_category.to_csv("consumer_category.csv")

In [16]:
##測試是否計算正確
unique_id=brand.id.unique()

selectdata=brand.loc[brand['id']== unique_id[898],].reset_index()
sumday=0
period=0
times=selectdata.shape[0]
if times==1:
    print('only once')
else:   
    for k in range(1,times):
        day1=datetime.datetime.strptime(selectdata['date'][k],'%Y/%m/%d')
        day2=datetime.datetime.strptime(selectdata['date'][k-1],'%Y/%m/%d')
        sumday=sumday+int((day1-day2).days)
        period=period+(int((day1-day2).days))/selectdata['quantity'][k-1]
    print("test:",sumday/(times-1))
print("loop:",consumer_brand.loc[898,'purchaseperiod_brand'])

test: 52.0
loop: 52.0


In [17]:
consumer_brand.head()

,id,brand,purchasetimes_brand,total_amount_brand,total_quantit_brand,purchaseperiod_brand,brandperiod,before_30day_time_brand,before_30day_quan_brand,before_30day_amount_brand,before_60day_time_brand,before_60day_quan_brand,before_60day_amount_brand,before_90day_time_brand,before_90day_quan_brand,before_90day_amount_brand,before_180day_time_brand,before_180day_quan_brand,before_180day_amount_brand
0,100007447,3718.0,1.0,2.69,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00
1,100053952,102504.0,7.0,23.03,7.0,57.666667,0.0,0.0,0.0,0.0,1.0,1.0,3.99,2.0,2.0,7.98,3.0,3.0,10.98
2,1000714152,13791.0,9.0,29.72,12.0,37.000000,0.0,0.0,0.0,0.0,1.0,1.0,2.50,2.0,2.0,6.79,5.0,6.0,17.28
3,100245416,3718.0,3.0,8.07,3.0,125.000000,0.0,0.0,0.0,0.0,1.0,1.0,2.69,1.0,1.0,2.69,2.0,2.0,5.38
4,100258679,3718.0,1.0,3.49,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00


In [20]:
##consumer_category測試是否計算正確
unique_id=category.id.unique()
selectdata=category.loc[category['id']== unique_id[919],].reset_index()
sumday=0
period=0
times=selectdata.shape[0]
if times==1:
    print('only once')
else:   
    for k in range(1,times):
        day1=datetime.datetime.strptime(selectdata['date'][k],'%Y/%m/%d')
        day2=datetime.datetime.strptime(selectdata['date'][k-1],'%Y/%m/%d')
        sumday=sumday+int((day1-day2).days)
        period=period+(int((day1-day2).days))/selectdata['quantity'][k-1]
    print("test:",sumday/(times-1))
print("loop:",consumer_category.loc[919,'purchaseperiod_category'])

test: 329.0
loop: 329.0


In [22]:
##consumer_company 測試是否計算正確
unique_id=company.id.unique()
selectdata=company.loc[company['id']== unique_id[55455],].reset_index()
sumday=0
period=0
times=selectdata.shape[0]
if times==1:
    print('only once')
else:   
    for k in range(1,times):
        day1=datetime.datetime.strptime(selectdata['date'][k],'%Y/%m/%d')
        day2=datetime.datetime.strptime(selectdata['date'][k-1],'%Y/%m/%d')
        sumday=sumday+int((day1-day2).days)
        period=period+(int((day1-day2).days))/selectdata['quantity'][k-1]
    print("test:",sumday/(times-1))
print("loop:",consumer_company.loc[55455,'purchaseperiod_company'])

only once
loop: 0.0


In [23]:
ccom=consumer_company.set_index('id')

In [24]:
ccate=consumer_category.set_index('id')


In [25]:
cbrand=consumer_brand.set_index('id')

In [27]:
cbrand.head()

,brand,purchasetimes_brand,total_amount_brand,total_quantit_brand,purchaseperiod_brand,brandperiod,before_30day_time_brand,before_30day_quan_brand,before_30day_amount_brand,before_60day_time_brand,before_60day_quan_brand,before_60day_amount_brand,before_90day_time_brand,before_90day_quan_brand,before_90day_amount_brand,before_180day_time_brand,before_180day_quan_brand,before_180day_amount_brand
id,,,,,,,,,,,,,,,,,,
100007447,3718.0,1.0,2.69,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00
100053952,102504.0,7.0,23.03,7.0,57.666667,0.0,0.0,0.0,0.0,1.0,1.0,3.99,2.0,2.0,7.98,3.0,3.0,10.98
1000714152,13791.0,9.0,29.72,12.0,37.000000,0.0,0.0,0.0,0.0,1.0,1.0,2.50,2.0,2.0,6.79,5.0,6.0,17.28
100245416,3718.0,3.0,8.07,3.0,125.000000,0.0,0.0,0.0,0.0,1.0,1.0,2.69,1.0,1.0,2.69,2.0,2.0,5.38
100258679,3718.0,1.0,3.49,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00


In [28]:
cc=ccom.join(ccate)
cc=cc.join(cbrand)
cc=cc.fillna(0)

In [41]:
cc[cc.before_90day_amount_company !=0].head()

,company,purchasetimes_company,total_amount_company,total_quantit_company,purchaseperiod_company,companyperiod,before_30day_time_company,before_30day_quan_company,before_30day_amount_company,before_60day_time_company,...,before_30day_amount_brand,before_60day_time_brand,before_60day_quan_brand,before_60day_amount_brand,before_90day_time_brand,before_90day_quan_brand,before_90day_amount_brand,before_180day_time_brand,before_180day_quan_brand,before_180day_amount_brand
id,,,,,,,,,,,,,,,,,,,,,
100007447,104460040.0,8.0,47.310000,9.0,52.000000,0.0,0.0,0.0,0.00,1.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00
100033247,104460040.0,13.0,71.509999,15.0,28.000000,0.0,0.0,0.0,0.00,1.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00
100053952,107717272.0,7.0,23.030000,7.0,57.666667,0.0,0.0,0.0,0.00,1.0,...,0.0,1.0,1.0,3.99,2.0,2.0,7.98,3.0,3.0,10.98
1000714152,105100050.0,28.0,102.169999,85.0,13.518519,0.0,5.0,16.0,12.43,9.0,...,0.0,1.0,1.0,2.50,2.0,2.0,6.79,5.0,6.0,17.28
100172712,104460040.0,11.0,34.160000,13.0,30.700000,0.0,0.0,0.0,0.00,2.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00


In [47]:
traindata=trainconsumer.join(cc,rsuffix="_firstpurchase",on="id").fillna(0)

In [48]:
traindata[traindata['before_30day_amount_brand']!=0]

,id,total_amount,total_quantity,total_trips,diffcompany,diffcategory,diffbrand,chain,offer,market,...,before_30day_amount_brand,before_60day_time_brand,before_60day_quan_brand,before_60day_amount_brand,before_90day_time_brand,before_90day_quan_brand,before_90day_amount_brand,before_180day_time_brand,before_180day_quan_brand,before_180day_amount_brand
8,100444554,16818.919860,2580,100,438,294,522,15,1208251,9,...,8.670000,2.0,3.0,8.670000,2.0,3.0,8.670000,2.0,3.0,8.670000
10,1005207774,4451.569972,1567,31,291,261,288,46,1204576,15,...,5.000000,2.0,4.0,9.490000,3.0,6.0,14.490000,5.0,10.0,24.490000
19,1017618004,4041.609976,1517,90,226,240,267,46,1199256,15,...,12.180000,5.0,6.0,19.660000,8.0,11.0,36.620000,14.0,23.0,72.820000
50,103104842,2674.949965,717,102,217,180,235,15,1204576,9,...,5.760000,1.0,2.0,5.760000,1.0,2.0,5.760000,1.0,2.0,5.760000
58,103394300,6546.649936,1245,110,240,263,316,14,1208251,8,...,8.480000,1.0,2.0,8.480000,1.0,2.0,8.480000,1.0,2.0,8.480000
83,104480050,3144.589971,1133,62,168,156,185,15,1204822,9,...,3.000000,1.0,1.0,3.000000,1.0,1.0,3.000000,1.0,1.0,3.000000
91,104884542,4015.199940,1361,132,206,170,227,4,1204576,1,...,3.000000,2.0,2.0,6.990000,4.0,4.0,12.370000,4.0,4.0,12.370000
116,1059512826,1298.389991,892,22,117,102,108,46,1199258,15,...,2.000000,2.0,2.0,4.000000,2.0,2.0,4.000000,4.0,4.0,9.000000
124,106508086,3956.949965,1302,58,268,235,304,4,1204576,1,...,3.000000,1.0,1.0,3.000000,2.0,2.0,7.490000,3.0,3.0,10.480000
154,1080588152,6383.510012,2140,156,282,288,322,214,1208329,8,...,19.850000,4.0,7.0,26.520000,7.0,12.0,45.520000,7.0,12.0,45.520000


In [50]:
traindata.shape

(160059, 72)

In [51]:
traindata.columns

Index(['id', 'total_amount', 'total_quantity', 'total_trips', 'diffcompany',
       'diffcategory', 'diffbrand', 'chain', 'offer', 'market', 'repeattrips',
       'repeater', 'offerdate', 'category', 'quantity', 'company',
       'offervalue', 'brand', 'company_firstpurchase', 'purchasetimes_company',
       'total_amount_company', 'total_quantit_company',
       'purchaseperiod_company', 'companyperiod', 'before_30day_time_company',
       'before_30day_quan_company', 'before_30day_amount_company',
       'before_60day_time_company', 'before_60day_quan_company',
       'before_60day_amount_company', 'before_90day_time_company',
       'before_90day_quan_company', 'before_90day_amount_company',
       'before_180day_time_company', 'before_180day_quan_company',
       'before_180day_amount_company', 'category_firstpurchase',
       'purchasetimes_category', 'total_amount_category',
       'total_quantit_category', 'purchaseperiod_category', 'categoryperiod',
       'before_30day_time_ca

In [52]:
list=['category','brand','company']
for k in list:
    col=k+'_firstpurchase'
    for i in range(traindata.shape[0]):
        if traindata.loc[i,col]==0:
            traindata.loc[i,col]=1
        else:
            traindata.loc[i,col]=0

In [53]:
traindata.to_csv('sum_consmer.csv')

In [54]:
consumer=pd.read_csv('sum_consmer.csv')
consumer['repeater'] = 1 * (consumer['repeater'] == 't')


In [55]:
consumer.head()

,Unnamed: 0,id,total_amount,total_quantity,total_trips,diffcompany,diffcategory,diffbrand,chain,offer,...,before_30day_amount_brand,before_60day_time_brand,before_60day_quan_brand,before_60day_amount_brand,before_90day_time_brand,before_90day_quan_brand,before_90day_amount_brand,before_180day_time_brand,before_180day_quan_brand,before_180day_amount_brand
0,0,100051423,2771.429977,807,85,190,192,228,4,1197502,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,100084808,2790.199984,907,74,221,213,249,20,1197502,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,100172712,7690.499942,3349,222,320,317,381,95,1200581,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,100200838,3880.729944,1001,61,192,170,222,17,1208251,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1002116097,4158.509984,1612,61,206,228,266,46,1197502,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
#consumer=consumer.drop(columns=['Unnamed: 0'])
consumer.shape

(160059, 72)

In [62]:
for i in range(consumer.shape[0]):
    if consumer.loc[i,'total_amount_company']>1000 or consumer.loc[i,'total_amount_company']<0:
        consumer.loc[i,'total_amount_company']=consumer['total_amount_company'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'total_quantit_company']>1000 or consumer.loc[i,'total_quantit_company']<0:
        consumer.loc[i,'total_quantit_company']=consumer['total_quantit_company'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'total_amount_category']>1000 or consumer.loc[i,'total_amount_category']<0:
        consumer.loc[i,'total_amount_category']=consumer['total_amount_category'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'total_quantit_category']>1000 or consumer.loc[i,'total_quantit_category']<0:
        consumer.loc[i,'total_quantit_category']=consumer['total_quantit_category'].mean()   
for i in range(consumer.shape[0]):
    if consumer.loc[i,'total_amount_brand']>1000 or consumer.loc[i,'total_amount_brand']<0:
        consumer.loc[i,'total_amount_brand']=consumer['total_amount_brand'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'total_quantit_brand']>1000 or consumer.loc[i,'total_quantit_brand']<0:
        consumer.loc[i,'total_quantit_brand']=consumer['total_quantit_brand'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'total_amount']>10000 or consumer.loc[i,'total_amount']<0:
        consumer.loc[i,'total_amount']=consumer['total_amount'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'total_quantity']>10000 or consumer.loc[i,'total_quantity']<0:
        consumer.loc[i,'total_quantity']=consumer['total_quantity'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'categoryperiod']<0 :
        consumer.loc[i,'categoryperiod']=0
for i in range(consumer.shape[0]):
    if consumer.loc[i,'brandperiod']<0:
        consumer.loc[i,'brandperiod']=0
for i in range(consumer.shape[0]):
    if consumer.loc[i,'companyperiod']<0:
        consumer.loc[i,'companyperiod']=0   
for i in range(consumer.shape[0]):
    if consumer.loc[i,'purchaseperiod_brand']<=0:
        consumer.loc[i,'purchaseperiod_brand']=consumer['purchaseperiod_brand'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'purchaseperiod_company']<=0:
        consumer.loc[i,'purchaseperiod_company']=consumer['purchaseperiod_company'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'purchaseperiod_category']<=0:
        consumer.loc[i,'purchaseperiod_category']=consumer['purchaseperiod_category'].mean()


In [65]:
consumer['ave_spend']=consumer['total_amount']/consumer['total_trips']
consumer['ave_quant']=consumer['total_quantity']/consumer['total_trips']
consumer['ave_unique_company_bought']=consumer['diffcompany']/consumer['total_trips']
consumer['ave_unique_category_bought']=consumer['diffcategory']/consumer['total_trips']
consumer['ave_unique_brand_bought']=consumer['diffbrand']/consumer['total_trips']

In [68]:
consumer[consumer.before_180day_amount_brand!=0].head()

,id,total_amount,total_quantity,total_trips,diffcompany,diffcategory,diffbrand,chain,offer,market,...,before_90day_quan_brand,before_90day_amount_brand,before_180day_time_brand,before_180day_quan_brand,before_180day_amount_brand,ave_spend,ave_quant,ave_unique_company_bought,ave_unique_category_bought,ave_unique_brand_bought
8,100444554,5673.575517,2580.0,100,438,294,522,15,1208251,9,...,3.0,8.67,2.0,3.0,8.67,56.735755,25.800000,4.380000,2.940000,5.220000
10,1005207774,4451.569972,1567.0,31,291,261,288,46,1204576,15,...,6.0,14.49,5.0,10.0,24.49,143.599031,50.548387,9.387097,8.419355,9.290323
19,1017618004,4041.609976,1517.0,90,226,240,267,46,1199256,15,...,11.0,36.62,14.0,23.0,72.82,44.906778,16.855556,2.511111,2.666667,2.966667
24,101961875,3397.119954,926.0,67,173,169,187,15,1208251,9,...,2.0,4.68,1.0,2.0,4.68,50.703283,13.820896,2.582090,2.522388,2.791045
26,1021253705,1061.249991,344.0,18,116,114,119,46,1199256,15,...,0.0,0.00,2.0,2.0,5.87,58.958333,19.111111,6.444444,6.333333,6.611111


In [70]:
consumer.drop_duplicates()
consumer.shape

(160059, 77)

In [71]:
consumer.to_csv('sum_consmer.csv')